In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics opencv-python --quiet

In [ ]:
import torch
print("GPU Available:", torch.cuda.is_available())

In [ ]:
import os

# Define Kaggle dataset path
src_path = "/kaggle/input/traffic-detection-project"
data_yaml = f"{src_path}/data.yaml"

# Read and fix paths
with open(data_yaml, 'r') as f:
    yaml_data = f.read()

yaml_data = yaml_data.replace('/kaggle/input/traffic-detection-project/train', f'{src_path}/train/images')
yaml_data = yaml_data.replace('/kaggle/input/traffic-detection-project/valid', f'{src_path}/valid/images')
yaml_data = yaml_data.replace('/kaggle/input/traffic-detection-project/test', f'{src_path}/test/images')

# Save fixed data.yaml to current directory
with open('data.yaml', 'w') as f:
    f.write(yaml_data)

print("✅ data.yaml fixed and saved.")

In [ ]:
import os

image_dir = "/kaggle/input/traffic-detection-project/train/images"
label_dir = "/kaggle/input/traffic-detection-project/train/labels"

images = set(os.path.splitext(f)[0] for f in os.listdir(image_dir))
labels = set(os.path.splitext(f)[0] for f in os.listdir(label_dir))

print("Unmatched Images:", images - labels)
print("Unmatched Labels:", labels - images)


In [ ]:
empty_labels = []
for f in os.listdir(label_dir):
    full_path = os.path.join(label_dir, f)
    if os.path.getsize(full_path) == 0:
        empty_labels.append(f)

print("Empty label files:", empty_labels)


In [ ]:
import os

image_dir = "/kaggle/input/traffic-detection-project/train/images"
label_dir = "/kaggle/input/traffic-detection-project/train/labels"

clean_image_dir = "/kaggle/working/clean_dataset/train/images"
clean_label_dir = "/kaggle/working/clean_dataset/train/labels"
os.makedirs(clean_image_dir, exist_ok=True)
os.makedirs(clean_label_dir, exist_ok=True)

# Remove empty labels and their images
for label_file in os.listdir(label_dir):
    label_path = os.path.join(label_dir, label_file)
    image_file = label_file.replace('.txt', '.jpg')  # or .png based on your dataset
    image_path = os.path.join(image_dir, image_file)

    if os.path.getsize(label_path) > 0:
        # Copy only valid ones
        os.system(f'cp "{label_path}" "{clean_label_dir}/{label_file}"')
        os.system(f'cp "{image_path}" "{clean_image_dir}/{image_file}"')

print("✅ Clean dataset created.")


In [ ]:
import os

def clean_yolo_folder(image_dir, label_dir, output_root):
    clean_image_dir = os.path.join(output_root, 'images')
    clean_label_dir = os.path.join(output_root, 'labels')
    os.makedirs(clean_image_dir, exist_ok=True)
    os.makedirs(clean_label_dir, exist_ok=True)

    for label_file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, label_file)
        image_file = label_file.replace('.txt', '.jpg')  # change if using .png
        image_path = os.path.join(image_dir, image_file)

        if os.path.getsize(label_path) > 0 and os.path.exists(image_path):
            os.system(f'cp "{label_path}" "{clean_label_dir}/{label_file}"')
            os.system(f'cp "{image_path}" "{clean_image_dir}/{image_file}"')

# Clean each split
base = "/kaggle/input/traffic-detection-project"
clean_base = "/kaggle/working/clean_data"

clean_yolo_folder(f"{base}/train/images", f"{base}/train/labels", f"{clean_base}/train")
clean_yolo_folder(f"{base}/valid/images", f"{base}/valid/labels", f"{clean_base}/valid")
clean_yolo_folder(f"{base}/test/images", f"{base}/test/labels", f"{clean_base}/test")

print("✅ Clean dataset ready at /kaggle/working/clean_data")



In [ ]:
yaml_text = f"""
train: {clean_base}/train/images
val: {clean_base}/valid/images
test: {clean_base}/test/images

nc: 5
names: ['bicycle', 'bus', 'car', 'motorbike', 'person']
"""

with open("data.yaml", "w") as f:
    f.write(yaml_text)

print("✅ Updated data.yaml saved.")


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(data="data.yaml", epochs=30, imgsz=640)


In [ ]:
model.val()

In [ ]:
import os

# List all training run folders
runs = os.listdir("runs/detect")
print("Training runs:", runs)

# List weights inside the latest one
weights_path = "runs/detect/train/weights"
if os.path.exists(weights_path):
    print("✅ Weights folder found. Contents:", os.listdir(weights_path))
else:
    print("❌ Weights folder not found!")


In [ ]:
import os

train_dirs = os.listdir("runs/detect")
print("Training runs found:", train_dirs)



In [ ]:
import os

# Loop through all runs and check if best.pt exists
for d in ['train4', 'train2', 'train', 'train3']:
    pt_path = f"runs/detect/{d}/weights/best.pt"
    if os.path.exists(pt_path):
        print(f"✅ Found model at: {pt_path}")

In [ ]:
import shutil

shutil.copy("runs/detect/train/weights/best.pt", "/kaggle/working/best.pt")
print("✅ Model is now available in /kaggle/working/best.pt")


In [ ]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("best.pt")

# Predict on an image
results = model.predict(source="/kaggle/input/traffic-detection-project/test/images/aguanambi-3390_png_jpg.rf.e162de23d695e4152f1d15c07646418e.jpg", save=True)

# Show where prediction was saved
print("🖼️ Prediction saved to:", results[0].save_dir)


In [ ]:
from IPython.display import Image, display
import os

# Change 'image1.jpg' to your image name
predicted_image_path = os.path.join(results[0].save_dir, "aguanambi-3390_png_jpg.rf.e162de23d695e4152f1d15c07646418e.jpg")
display(Image(filename=predicted_image_path))

In [ ]:
pip install ultralytics opencv-python deep_sort_realtime